In [ ]:
!pip install numpy Pillow Matplotlib pandas seaborn scikit-learn requests scikit-image mlxtend prefixspan scikit-fuzzy umap-learn openpyxl stanza torch torchvision tesseract pytesseract nltk wordcloud spacy tensorflow==2.19.0 tensorflow-datasets opencv-python ucimlrepo nbconvert

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 MB 28.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 36.2 MB/s eta 0:00:00
  Created wheel for prefixspan: filename=prefixspan-0.5.2-py3-none-any.whl size=11215 sha256=ec69a21566a0e33708a786c7513fb8ef6882940d7679208b15afa5d883125d61
  Stored in directory: /root/.cache/pip/wheels/72/36/65/f9e2bbe4998ab4cdf6e7a015f38bdbb6925301dcf2cce34a88
  Created wheel for tesseract: filename=tesseract-0.1.3-py3-none-any.whl size=45562552 sha256=60fe11410ec2a369ae8bed910a78cd59e5bdf727ed5d513792db64a89dda6b42
  Stored in directory: /root/.cache/pip/wheels/13/1f/8e/2d6c0e358fd6d01ca80ecd9185a374bcda358

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

In [ ]:
# con esto le ensenho al modelo
x = np.array([[[0], [1], [2]], [[1], [2], [3]]]) # secuencia

# esto es lo que quiero que mi modelo trate de predecir
y = np.array([3, 4])

# defino mi modelo de rnn simple
model = Sequential([
  SimpleRNN(10, activation="tanh", input_shape=(3, 1)),
  Dense(1)
])

model.compile(optimizer="adam", loss="MSE")
model.fit(x, y, epochs=200, verbose=0)

print("Prediccion del modelo para [2, 3, 4]: ", model.predict(np.array([[[2], [3], [4]]])))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
Prediccion del modelo para [2, 3, 4]:  [[2.7801075]]


Caso de uso para prediccion de temperaturas

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# traemos el dataset
df = pd.read_csv(r"temperature.csv")

temps = df["San Francisco"].dropna().values.reshape(-1, 1)

# Normalizacion
scaler = MinMaxScaler()
temps_scaled = scaler.fit_transform(temps)

x, y = [], []

for i in range(len(temps_scaled) - 25):
	x.append(temps_scaled[i:i+24])
	y.append(temps_scaled[i+24])

x, y = np.array(x), np.array(y)

model = Sequential([
	SimpleRNN(50, activation="tanh", input_shape=(24, 1)),
	Dense(1)
])

model.compile(optimizer="adam", loss="mse")
model.fit(x, y, epochs=10, batch_size=64)

pred = model.predict(x[:1])
print("Prediccion para la proxima hora: ", scaler.inverse_transform(pred))

Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


695/695 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.0162
Epoch 2/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 9.2568e-04
Epoch 3/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 6.4794e-04
Epoch 4/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 5.3661e-04
Epoch 5/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 4.4994e-04
Epoch 6/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 4.2118e-04
Epoch 7/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 4.2593e-04
Epoch 8/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 4.1292e-04
Epoch 9/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 4.0726e-04
Epoch 10/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 4.1316e-04
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
Prediccion para la proxima hora:  [[289.15237]]


In [ ]:
pred = model(x[:1])
pred_kelvin = scaler.inverse_transform(pred)[0][0]
pred_celsius = pred_kelvin - 273.15
print(f"Prediccion de la siguiente hora: {pred_kelvin:.2f} K / {pred_celsius:.2f}ºC")

Prediccion de la siguiente hora: 289.15 K / 16.00ºC
